In [20]:
import pandas as pd
import os
import fnmatch
from PIL import Image
from transformers import (
    ViTImageProcessor ,
    ViTModel,
    ImageClassificationPipeline
)
from dotenv import load_dotenv
from qdrant_client import QdrantClient, models
from qdrant_client.models import VectorParams, Distance
load_dotenv()
pretrained_model_name = r"..\trained_model"

In [21]:

base_dir = "Part1"
all_images = os.listdir(base_dir)
len(all_images)

300

In [22]:

all_r_images = [f for f in all_images if fnmatch.fnmatch(f, '*R_*.tiff')]

In [23]:
all_train_images = list(map(lambda item: f"{base_dir}/{item}", all_r_images))
len(all_train_images)

150

In [24]:
all_train_images[:5]

['Part1/001_01_R_01.tiff',
 'Part1/001_01_R_02.tiff',
 'Part1/001_01_R_03.tiff',
 'Part1/001_01_R_04.tiff',
 'Part1/001_01_R_05.tiff']

In [25]:
df_metadata = pd.read_excel("./Data-information.xlsx")
df_metadata.head()

,ID,Gender,Brand,Model/Details,Size
0,001_01,W,Converse,NaN,5.5
1,002_01,M,Saucony,cohesion_xt600,9.5
2,002_02,M,Saucony,ride5,9.5
3,002_05,M,Saucony,stratos4,9.5
4,002_06,M,Saucony,phantom,9.5


In [26]:
# Create a dictionary to store ID to filepath mapping
image_dict = {}

# Process each image path
for path in all_train_images:
    # Extract the ID from the filename (e.g., '001_01' from '001_01_R_05.tiff')
    filename = os.path.basename(path)  # Get filename without path
    id_part = filename.split('_R_')[0]  # Split on '_R_' and take first part
    print(id_part)
    
    # Add to dictionary (if multiple images exist for same ID, keep the last one)
    image_dict[id_part] = path

# Add image_path column to DataFrame
df_metadata['image_url'] = df_metadata['ID'].map(image_dict)

001_01
001_01
001_01
001_01
001_01
002_01
002_01
002_01
002_01
002_01
002_02
002_02
002_02
002_02
002_02
002_05
002_05
002_05
002_05
002_05
002_06
002_06
002_06
002_06
002_06
002_07
002_07
002_07
002_07
002_07
003_01
003_01
003_01
003_01
003_01
003_02
003_02
003_02
003_02
003_02
003_03
003_03
003_03
003_03
003_03
004_01
004_01
004_01
004_01
004_01
004_04
004_04
004_04
004_04
004_04
004_05
004_05
004_05
004_05
004_05
004_06
004_06
004_06
004_06
004_06
004_07
004_07
004_07
004_07
004_07
005_01
005_01
005_01
005_01
005_01
005_02
005_02
005_02
005_02
005_02
005_03
005_03
005_03
005_03
005_03
005_04
005_04
005_04
005_04
005_04
005_05
005_05
005_05
005_05
005_05
005_06
005_06
005_06
005_06
005_06
005_07
005_07
005_07
005_07
005_07
005_08
005_08
005_08
005_08
005_08
005_09
005_09
005_09
005_09
005_09
005_10
005_10
005_10
005_10
005_10
005_11
005_11
005_11
005_11
005_11
006_01
006_01
006_01
006_01
006_01
006_02
006_02
006_02
006_02
006_02
006_03
006_03
006_03
006_03
006_03
006_04
006_04
006_04

In [27]:
df_metadata.head()

,ID,Gender,Brand,Model/Details,Size,image_url
0,001_01,W,Converse,NaN,5.5,Part1/001_01_R_05.tiff
1,002_01,M,Saucony,cohesion_xt600,9.5,Part1/002_01_R_05.tiff
2,002_02,M,Saucony,ride5,9.5,Part1/002_02_R_05.tiff
3,002_05,M,Saucony,stratos4,9.5,Part1/002_05_R_05.tiff
4,002_06,M,Saucony,phantom,9.5,Part1/002_06_R_05.tiff


In [28]:
df_metadata = df_metadata.dropna(subset=['image_url'])

In [29]:
len(df_metadata)

30

In [30]:
images = list(map(lambda item: Image.open(item), df_metadata['image_url']))

In [31]:

processor  = ViTImageProcessor.from_pretrained(pretrained_model_name)
model = ViTModel.from_pretrained(
    pretrained_model_name,
)

Some weights of ViTModel were not initialized from the model checkpoint at ..\trained_model and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
inputs = processor(images, return_tensors="pt")

# Pass through ViT model
outputs = model(**inputs)

# Extract embeddings from the last hidden state
embeddings = outputs.last_hidden_state[:, 0, :]  # Take CLS token embedding

# Convert to list
embeddings = embeddings.detach().numpy().tolist()

In [33]:
qclient = QdrantClient(
    url=os.getenv("QDRANT_CLUSTER"),
    api_key=os.getenv("QDRANT_API_KEY")
)

qclient

In [34]:
collection_name = "shoeprints_part1"

collection = qclient.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=len(embeddings[0]),
        distance = Distance.COSINE
    )
)

C:\Users\Varun\AppData\Local\Temp\ipykernel_14856\3540345574.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  collection = qclient.recreate_collection(


In [35]:
payload = df_metadata.to_dict(orient='records')
payload

[{'ID': '001_01',
  'Gender': 'W',
  'Brand': 'Converse',
  'Model/Details': nan,
  'Size': 5.5,
  'image_url': 'Part1/001_01_R_05.tiff'},
 {'ID': '002_01',
  'Gender': 'M',
  'Brand': 'Saucony',
  'Model/Details': 'cohesion_xt600',
  'Size': 9.5,
  'image_url': 'Part1/002_01_R_05.tiff'},
 {'ID': '002_02',
  'Gender': 'M',
  'Brand': 'Saucony',
  'Model/Details': 'ride5',
  'Size': 9.5,
  'image_url': 'Part1/002_02_R_05.tiff'},
 {'ID': '002_05',
  'Gender': 'M',
  'Brand': 'Saucony',
  'Model/Details': 'stratos4',
  'Size': 9.5,
  'image_url': 'Part1/002_05_R_05.tiff'},
 {'ID': '002_06',
  'Gender': 'M',
  'Brand': 'Saucony',
  'Model/Details': 'phantom',
  'Size': 9.5,
  'image_url': 'Part1/002_06_R_05.tiff'},
 {'ID': '002_07',
  'Gender': 'M',
  'Brand': 'Saucony',
  'Model/Details': 'guide9',
  'Size': 9.5,
  'image_url': 'Part1/002_07_R_05.tiff'},
 {'ID': '003_01',
  'Gender': 'M',
  'Brand': 'BAGO',
  'Model/Details': 'extralight',
  'Size': 10.0,
  'image_url': 'Part1/003_01_R_05

In [36]:
records = [
    models.Record(
        id = idx,
        payload = payload[idx],
        vector = embeddings[idx]
    )
    for idx, _ in enumerate(payload)
]

In [37]:
qclient.upload_records(collection_name, records)

C:\Users\Varun\AppData\Local\Temp\ipykernel_14856\660385042.py:1: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qclient.upload_records(collection_name, records)
